In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# First cell - Imports and Setup
import torch
import os
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from tqdm.notebook import tqdm  # Use tqdm.notebook for Kaggle
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, f1_score
from PIL import Image

# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset path for Kaggle
DATASET_PATH = "../input/dataset-frame/Dataset_frame"  # Adjust based on your dataset path
print(f"Dataset path exists: {os.path.exists(DATASET_PATH)}")

In [ ]:
# Second cell - Dataset and Model Classes
class DeepfakeDataset(Dataset):
    def __init__(self, root_dir, split='Train', transform=None):
        """
        Args:
            root_dir: Base directory of the dataset
            split: 'Train', 'Validation', or 'Test'
            transform: Optional transforms
        """
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Verify directory structure
        print(f"Loading {split} data from: {self.root_dir}")
        
        # Load both Real and Fake images
        for label, class_name in enumerate(['Real', 'Fake']):
            class_dir = os.path.join(self.root_dir, class_name)
            if not os.path.exists(class_dir):
                raise RuntimeError(f"Directory not found: {class_dir}")
            
            files = os.listdir(class_dir)
            print(f"Found {len(files)} {class_name} images")
            
            for img_name in files:
                self.images.append(os.path.join(class_dir, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        try:
            image = Image.open(img_path).convert('RGB')
            label = self.labels[idx]
            
            if self.transform:
                image = self.transform(image)
                
            return image, torch.tensor(label, dtype=torch.float32)
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            # Return a default tensor in case of error
            return torch.zeros((3, 224, 224)), torch.tensor(0., dtype=torch.float32)

def get_transforms(is_training=True):
    if is_training:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                              std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                              std=[0.229, 0.224, 0.225])
        ])

class DeepfakeDetector(nn.Module):
    def __init__(self):
        super(DeepfakeDetector, self).__init__()
        # Use EfficientNet-B3 with pretrained weights
        self.base_model = models.efficientnet_b3(weights='IMAGENET1K_V1')
        num_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Identity()
        
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 1)
        )
    
    def forward(self, x):
        features = self.base_model(x)
        return self.classifier(features)

In [ ]:
def train_model(config):
    # Initialize model and move to GPU
    model = DeepfakeDetector().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max', 
        factor=0.1, 
        patience=3
    )
    
    # Initialize datasets
    try:
        train_dataset = DeepfakeDataset(
            config['data_dir'], 
            "Train", 
            transform=get_transforms(True)
        )
        val_dataset = DeepfakeDataset(
            config['data_dir'], 
            "Validation", 
            transform=get_transforms(False)
        )
    except Exception as e:
        print(f"Error initializing datasets: {str(e)}")
        return

    # Create data loaders with updated num_workers
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=2,
        pin_memory=True,
        persistent_workers=True  # Added for better performance
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=2,
        pin_memory=True,
        persistent_workers=True  # Added for better performance
    )

    # Initialize training variables
    best_auc = 0.0
    early_stopping_counter = 0
    
    # Updated GradScaler initialization
    scaler = torch.amp.GradScaler('cuda')

    # Training loop
    for epoch in range(config['num_epochs']):
        # Training phase
        model.train()
        train_losses = []
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config["num_epochs"]}')
        
        for images, labels in progress_bar:
            images = images.to(device, non_blocking=True)  # Added non_blocking=True
            labels = labels.to(device, non_blocking=True)  # Added non_blocking=True
            
            optimizer.zero_grad(set_to_none=True)  # More efficient than zero_grad()
            
            # Updated autocast implementation
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(images).squeeze()
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_losses.append(loss.item())
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

        # Validation phase
        model.eval()
        val_losses = []
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc='Validation'):
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                
                # Updated autocast implementation
                with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                    outputs = model(images).squeeze()
                    loss = criterion(outputs, labels)
                
                val_losses.append(loss.item())
                val_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        # Calculate metrics
        val_auc = roc_auc_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, np.array(val_preds) > 0.5, average='weighted')
        
        print(f'\nEpoch {epoch+1}:')
        print(f'Train Loss: {np.mean(train_losses):.4f}')
        print(f'Val Loss: {np.mean(val_losses):.4f}')
        print(f'Val AUC: {val_auc:.4f}')
        print(f'Val F1: {val_f1:.4f}')

        # Update learning rate
        scheduler.step(val_auc)

        # Save best model
        if val_auc > best_auc:
            best_auc = val_auc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_auc': best_auc,
                'scaler_state_dict': scaler.state_dict()
            }, '/kaggle/working/best_model.pth')
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        # Early stopping
        if early_stopping_counter >= config['early_stopping_patience']:
            print(f'\nEarly stopping triggered after epoch {epoch+1}')
            break

    print(f'\nBest validation AUC: {best_auc:.4f}')
    return model

# Update the configuration with optimized parameters
config = {
    'data_dir': DATASET_PATH,
    'batch_size': 64,  # You can try 64 if memory allows
    'learning_rate': 1e-4,
    'num_epochs': 20,
    'early_stopping_patience': 5
}

In [ ]:
# Add memory optimization before training
torch.backends.cudnn.benchmark = True  # Enable cudnn autotuner
torch.backends.cuda.matmul.allow_tf32 = True  # Allow TF32 on Ampere
torch.backends.cudnn.allow_tf32 = True  # Allow TF32 on Ampere

# Train the model
model = train_model(config)

In [ ]:
def test_model(model_path, data_dir):
    test_dataset = DeepfakeDataset(data_dir, "Test", transform=get_transforms(False))
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    model = DeepfakeDetector().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    test_preds, test_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            outputs = model(images).squeeze()
            test_preds.extend(torch.sigmoid(outputs).cpu().numpy())
            test_labels.extend(labels.numpy())

    auc_score = roc_auc_score(test_labels, test_preds)
    print(f"Test AUC-ROC Score: {auc_score:.4f}")

if __name__ == "__main__":
    test_model("/kaggle/working/best_model.pth", DATASET_PATH)
